In [2]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from firebase import FirebaseManager 

In [4]:
def retrieve_professions_from_field_first(field):
    h3_elements = driver.find_elements(By.XPATH, "//h3[contains(text(), 'Professions')]")
    print(h3_elements)
    if len(h3_elements)==0:
        raise Exception(f"Aucun element trouvé pour la première fonction pour {field}")
    for h3_element in h3_elements:
        ul_element = h3_element.find_element(By.XPATH, "following-sibling::ul")
        driver.execute_script("arguments[0].scrollIntoView(true);", h3_element)
        if ul_element:
            li_elements = ul_element.find_elements(By.XPATH, "./li")
            for element in li_elements:
                text = driver.execute_script("return arguments[0].innerText;", element)
                print(text)
                if text not in data[field]:
                    datas[field].append(text)
        time.sleep(0.5)
    # driver.back()



def retrieve_professions_from_field_second(field):
    strong_elements = driver.find_elements(By.XPATH, "//p[strong[contains(text(), 'Professions')]]")
    print(f"strong: {strong_elements}")
    if len(strong_elements)==0:
        errors.append(field)
        raise Exception(f"Aucun element trouvé pour la deuxième fonction pour {field}")
    for strong_element in strong_elements:
        ul_element = strong_element.find_element(By.XPATH, "following-sibling::ul")
        if ul_element:
            driver.execute_script("arguments[0].scrollIntoView(true);", ul_element)
            li_elements = ul_element.find_elements(By.XPATH, "./li")
            for element in li_elements:
                text = driver.execute_script("return arguments[0].innerText;", element)
                print(text)
                if text not in data[field]:
                    datas[field].append(text)
        time.sleep(0.5)
    # driver.back()

def retrieve_professions_from_field(field):
    global nb_of_functions_tried, nb_of_functions
    try:
        if nb_of_functions_tried==1:
            retrieve_professions_from_field_first(field)
            driver.back()
        if nb_of_functions_tried==2:
            retrieve_professions_from_field_second(field)
            driver.back()
            nb_of_functions_tried=1
    except Exception as e:
        print(e)
        nb_of_functions_tried+=1
        retrieve_professions_from_field(field)

    if nb_of_functions_tried > nb_of_functions:
        driver.back()
        nb_of_functions_tried=1
    return



In [5]:
driver = webdriver.Chrome()
driver.get("https://www.ulaval.ca/etudes/programmes")
title = driver.title

# Keys represente fields and values are arrays whose contains professions 
datas={}
errors=[]
nb_of_functions=2
nb_of_functions_tried=1

pagination_next_element=driver.find_element(By.ID,"pagination-heading").find_element(By.XPATH, "following-sibling::ul").find_element(By.XPATH, "./li[@class='pagination-item pagination-item--suivant']")
print(pagination_next_element)
while pagination_next_element:
    results_element= driver.find_element(By.ID,"resultats")
    children=results_element.find_elements(By.XPATH, "./ul[@class='views-row']")

    for child in children:
        element=child.find_element(By.XPATH, "./li/a")
        field=element.find_element(By.XPATH,"./div[@class='main-info']/h3[@class='titre-programme']").text
        datas[field]=[]
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        time.sleep(0.5)
        driver.execute_script("arguments[0].click();", element)
        retrieve_professions_from_field(field)
    
    driver.execute_script("arguments[0].scrollIntoView(true);", pagination_next_element)
    time.sleep(0.5)
    driver.execute_script("arguments[0].click();", pagination_next_element.find_element(By.TAG_NAME, "a"))
    try:
        pagination_next_element=driver.find_element(By.ID,"pagination-heading").find_element(By.XPATH, "following-sibling::ul").find_element(By.XPATH, "./li[@class='pagination-item pagination-item--suivant']")
    except Exception as e:
        pagination_next_element=None
   

print(datas)


<selenium.webdriver.remote.webelement.WebElement (session="edcde7313c920c8453a3a879babde6f0", element="f.44660B7A3B5DB4430156527677609DDB.d.D4D4CCBA35FB7481548EAA7FFEBC2276.e.92")>
[<selenium.webdriver.remote.webelement.WebElement (session="edcde7313c920c8453a3a879babde6f0", element="f.44660B7A3B5DB4430156527677609DDB.d.79CFBF6097B0253977A8DEE7121C597F.e.129")>]
Actuaire
name 'data' is not defined
strong: []
Aucun element trouvé pour la deuxième fonction pour actuariat
[]
Aucun element trouvé pour la première fonction pour administration des affaires
strong: [<selenium.webdriver.remote.webelement.WebElement (session="edcde7313c920c8453a3a879babde6f0", element="f.44660B7A3B5DB4430156527677609DDB.d.3739C0F710FDA82C7178AB9DA6824243.e.156")>, <selenium.webdriver.remote.webelement.WebElement (session="edcde7313c920c8453a3a879babde6f0", element="f.44660B7A3B5DB4430156527677609DDB.d.3739C0F710FDA82C7178AB9DA6824243.e.157")>, <selenium.webdriver.remote.webelement.WebElement (session="edcde7313

In [5]:
collection_names=["professions_by_field","posts_by_field"]
credential_path="./indeedrecommender-firebase-adminsdk-3zci4-966775593c.json"
firebase_manager=FirebaseManager(credential_path)
#for key in datas.keys():
firebase_manager.add_data_to_firestore(collection_names[0],"key","datas[key]")
#

Error adding data to Firestore: 'str' object has no attribute 'items'
